## Creates the `lahinch_surf` feature view, trains the `cjsurf_model`

This training pipeline can be in Google Colab or a Jupyter notebook.

### Requirements

 * hopsworks

In [ ]:
#!pip install -U hopsworks

In [ ]:
import hopsworks
from sklearn.neighbors import KNeighborsClassifier
import hsfs
import numpy as np
import joblib
import os

### Not app.hopsworks.ai ?

If you are running your own Hopsworks cluster (not app.hopsworks.ai):

 * uncomment the cell below
 * fill in details for your cluster
 * run the cel

In [ ]:
# Uncomment this cell and fill in details if you are running your own Hopsworks cluster
#!echo "xxxx" > api-key.txt
# key=""
# with open("api-key.txt", "r") as f:
#     key = f.read().rstrip()
# os.environ['HOPSWORKS_PROJECT']="cjsurf"
# os.environ['HOPSWORKS_HOST']="35.187.178.84"
# os.environ['HOPSWORKS_API_KEY']=key

### Connect to your Hopsworks cluster

If you only set the HOPSWORKS_API_KEY, it will assume you are connecting to app.hopsworks.ai.
Set HOPSWORKS_HOST and HOPSWORKS_PROJECT environment variables to connect to a different Hopsworks cluster.

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

### Online Transformations for input features

Normalize the numerical input features (`height`, `period`, `direction`) with HSFS built-in `standard_scalar`.

In [ ]:
hours=119

standard_scaler = fs.get_transformation_function(name="standard_scaler")

transformation_functions = {
    "height": standard_scaler,
    "period": standard_scaler,
    "direction": standard_scaler,
}
for i in range(1,hours):
    transformation_functions["height" + str(i*2)]=standard_scaler
    transformation_functions["period" + str(i*2)]=standard_scaler
    transformation_functions["direction" + str(i*2)]=standard_scaler    

## Select features for the Feature View

Join the `wave_height` from `lahinch` feature group with all of the features from the `swells_exploded` feature group. 

The `query` object can be inspected - `show` rows, `read` to get a DataFrame.

In [ ]:
lahinch_fg = fs.get_feature_group("lahinch", version=1)
swells_fg = fs.get_feature_group("swells_exploded", version=1)
query = lahinch_fg.select(['wave_height']).join(
    swells_fg.select_except(['beach_id'])) 

### Get or create the `lahinch_surf` feature view

In [ ]:
version=1
try: 
    fv = fs.get_feature_view("lahinch_surf", version=version)
except:
    fv = fs.create_feature_view(name='lahinch_surf', 
                            description="Lahinch surf height prediction features",
                            version=version,
                            labels=["wave_height"],
                            query=query)

In [ ]:
td_version=1

#X_train,y_train,X_test,y_test = fv.train_test_split(0.1)

### Create training data as files with the `lahinch_surf` feature view

This will run a Spark job on Hopsworks that will join the features together and write them out as a `csv` file in Hopsworks.

In [ ]:
td_version, td_job = fv.create_train_test_split(
    description = 'cjsurf training data',
    data_format = 'csv',
    test_size = 0.1,
    version=td_version,
    write_options = {'wait_for_job': True}
)

### Read your Training Data as Pandas DataFrames
You can read back the training data as Pandas Dataframes - here split into train/test sets, with `X_` storing the features and `y_` the labels.

In [ ]:
X_train, y_train, X_test, y_test = fv.get_train_test_split(td_version)

Not all of the features are needed for training - only the height, period, direction of the swells with a `hits_at` timestamp that is closest to, but before the observation of the `surf_height`.

In [ ]:
X_train = X_train[['height', 'period', 'direction']]
X_test = X_test[['height', 'period', 'direction']]
X_train

Encode the label (`wave_height`) in both the train and test DataFrames

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_encoded=le.fit_transform(y_train['wave_height'])
y_test_encoded=le.fit_transform(y_test['wave_height'])

## Train the model
Using k-nearest neighbors in Scikit-Learn

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train_encoded)

### Evaluate the Model

On our test set, calculate the classification scores: precision, recall, fscore.
It may also be useful to calculate the rmse to see how close all the predictions were to the true wave_heights.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error
%matplotlib inline

y_pred = model.predict(X_test)

precision, recall, fscore, _ = precision_recall_fscore_support(
    y_test_encoded, y_pred, average="weighted", zero_division=1)

rmse = mean_squared_error(y_test_encoded, y_pred)

# We model surf height prediction as a classification problem, as there
# only a small number of heights reported on Lahinch beach, but the RMSE
# is still a useful metric for evaluating if your model is more accurate or not
metrics = {
    "precision" : precision,
    "recall" : recall,
    "fscore" : fscore,
    "rmse" : rmse
}

error_rates = []
num_samples=len(y_pred)
for a in range(1, num_samples):
    error_rates.append(np.mean(y_test_encoded[a] - y_pred))

plt.plot(range(0,num_samples), y_pred, label="predicted")
plt.plot(range(0,num_samples), y_test_encoded, label="actual")
plt.title('Surf height predictions')
plt.xlabel('Day')
plt.ylabel('Height')
plt.legend()
plt.show()

## Register the model with Hopsworks Model Registry

The model registry stores your model for later use in inference.

Here, we also specify the model input/output schema using the DataFrame containing the training features.
We also send a dict containing the `metrics` computed in the last cell, as these will be shown for the model in the registry. 

We can also include a sample to enable a deployed model to be easily tested in the UI. The sample will be stored in the model registry, and when you create a deployment with the model, there is a `Test Model` UI button which will send that sample to the model for scoring, returning the result. It is useful for debugging models.

Here, we are creating a `mr.python` model for Scikit-Learn. There is also a `mr.tf` model available for TensorFlow models.
When you call `save()` on the model returned by HSML, it uploads the model and any artifacts to Hopsworks.


In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

project = hopsworks.login()

mr = project.get_model_registry()
    
joblib.dump(model,'knn_model.pkl')

input_example = X_train.sample()
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

knn_model = mr.python.create_model(
    version=1,
    name="cjsurf", 
    metrics=metrics,
    model_schema=model_schema,
    input_example=input_example, description="Lahinch Surf height predictions")


In [ ]:
%%writefile knn_model/cjsurf_predictor.py

import joblib
import os

class Predict(object):
    
    def __init__(self):
        # NOTE: env var ARTIFACT_FILES_PATH has the local path to the model artifact files        
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/knn_model.pkl")


    def predict(self, inputs):
        """ Serves a prediction request from a trained model"""
        return self.model.predict(inputs).tolist()

In [ ]:
knn_model.save(model_dir)

### Create and start a model deployment

You can now create a deployment from the model. 
Call `start()` on the model deployment to start it running on KServe.

In [ ]:
predictor_script_path = knn_model.version_path + "/cjsurf_predictor.py"

model_deployment = knn_model.deploy(name="cjsurf",
                                    model_server="PYTHON",
                                    script_file=predictor_script_path,
                                    serving_tool="KSERVE"
                                   )

In [ ]:
model_deployment.start()